# Chapter 1: Basic Prompt Structure

（第1章：基本的なプロンプト構造）

- [レッスン](#lesson)
- [演習](#exercises)
- [例のプレイグラウンド](#example-playground)

## Setup

次のセットアップセルを実行して、APIキーの読み込みと `get_completion` ヘルパー関数の準備を行います。

In [3]:
# %pip install anthropic --quiet

# Import the hints module from the utils package
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Import python's built-in regular expression library
import re
from anthropic import AnthropicBedrock

%store -r MODEL_NAME
%store -r AWS_REGION

client = AnthropicBedrock(aws_region=AWS_REGION)

def get_completion(prompt, system=''):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": prompt}
        ],
        system=system
    )
    return message.content[0].text

---

## Lesson

Anthropic には 2 つの API、旧式の [Text Completions API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-text-completion.html) と、現在の [Messages API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html) があります。このチュートリアルでは **Messages API のみ** を使用します。

Messages API で Claude を呼び出すには、最低限次のパラメータが必要です。
- `model`: 呼び出したいモデルの [APIモデル名](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html#model-ids-arns)

- `max_tokens`: 停止するまでに生成するトークン数の上限。Claude は上限に達する前に停止することもあります。この値はあくまで **最大値** を指定するもので、また *ハードストップ*（強制停止）なので、単語や文の途中で生成が途切れることがあります。

- `messages`: 入力メッセージの配列。モデルは `user` と `assistant` の会話ターンが交互に並ぶ形式で動作するよう学習されています。新しい `Message` を作る際、`messages` に過去の会話ターンを渡し、モデルは次の `Message` を生成します。
  - 各入力メッセージは `role` と `content` を持つオブジェクトである必要があります。`user` のみ1件でも良いですし、複数の `user` と `assistant` を含めることもできます（その場合は交互である必要があります）。最初のメッセージは必ず `user` の `role` で始めます。

また、次のような任意パラメータもあります。
- `system`: システムプロンプト（後述）。
  
- `temperature`: 応答のばらつき度合い。このレッスンと演習では `temperature` を 0 に設定しています。

APIパラメータの完全な一覧は、[APIドキュメント](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-claude.html) を参照してください。

### Examples

正しい形式のプロンプトに対して Claude がどのように応答するか見てみましょう。次の各セルを実行（`shift+enter`）すると、ブロックの下に Claude の応答が表示されます。

In [4]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

Hello! I'm doing well, thank you for asking. I'm here and ready to help with whatever you'd like to discuss or work on. How are you doing today?


In [6]:
# Prompt
PROMPT = "海の色を教えてください"
# PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

海の色は一般的に**青色**ですが、実際にはさまざまな要因によって色が変化します。

## 主な海の色とその理由

**青色（最も一般的）**
- 太陽光が海水に入ると、赤い光は吸収され、青い光が散乱・反射されるため

**その他の色**
- **緑色**：プランクトンや藻類が多い場合
- **灰色**：曇りの日や嵐の時
- **エメラルドグリーン**：浅い海や珊瑚礁周辺
- **深い紺色**：深海部分
- **茶色や黄色**：泥や砂が混じっている場合

## 色に影響する要因
- 水深
- 天候
- 海底の地形
- プランクトンの量
- 太陽の角度
- 波の状態

同じ海でも場所や時間、季節によって美しく色が変わるのが海の魅力の一つですね。


In [7]:
# Prompt
PROMPT = "Celine Dionが生まれた年を教えてください"
# PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

セリーヌ・ディオン（Celine Dion）は1968年に生まれました。正確には1968年3月30日にカナダのケベック州で生まれています。


次に、Messages API の正しいフォーマットを満たしていないプロンプト例を見てみましょう。こうした不正な形式のプロンプトでは、Messages API はエラーを返します。

まずは、`messages` 配列の中に `role` と `content` フィールドがない例です。

> ⚠️ **Warning:** `messages` パラメータの形式が誤っているため、次のセルはエラーを返します。これは期待どおりの挙動です。

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Claude, how are you?"}
        ]
    )

# Print Claude's response
print(response[0].text)

次は `user` と `assistant` の `role` が交互になっていない例です。

> ⚠️ **Warning:** `user` と `assistant` の `role` が交互ではないため、Claude はエラーを返します。これは期待どおりの挙動です。

In [12]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ]
    )

# Print Claude's response
print(response.content[0].text)

Celine Dion was born in 1968.

Here are some other notable facts about her:

**Career highlights:**
- She's a Canadian singer who became one of the best-selling music artists of all time
- Known for her powerful vocals and wide vocal range
- Gained international fame with songs like "My Heart Will Go On" (from Titanic), "The Power of Love," and "Because You Loved Me"
- Has won multiple Grammy Awards and has sold over 200 million records worldwide

**Personal background:**
- Born in Charlemagne, Quebec, Canada, the youngest of 14 children
- Started performing professionally as a child
- Married her manager René Angélil in 1994 (he was significantly older and had managed her career since she was young)
- Had three sons with René before he passed away in 2016

**Other notable aspects:**
- Performed in both French and English throughout her career
- Had a very successful Las Vegas residency that ran for many years
- Known for her emotional performances and connection with audiences
- Has b

In [11]:
response.stop_reason

'end_turn'

`user` と `assistant` のメッセージは **必ず交互** に並べ、かつメッセージは **必ず `user` ターンから開始** しなければなりません。`user` と `assistant` のペアを複数入れることで、複数ターンの会話をシミュレートできます。最後の `assistant` メッセージに文言を入れて、Claude に続きから書かせることもできます（詳細は後の章で扱います）。

#### System Prompts

**システムプロンプト** も利用できます。システムプロンプトは、"User" ターンで質問やタスクを提示する前に、Claude に **文脈・指示・ガイドライン** を与える方法です。

構造的には、システムプロンプトは `user` と `assistant` のメッセージ一覧とは別に扱われるため、`system` パラメータとして渡します（ノートブックの [Setup](#setup) セクションにある `get_completion` の構造を確認してください）。

このチュートリアルでは、システムプロンプトを使える場面では `get_completion` に `system` 引数を用意しています。システムプロンプトを使わない場合は、`SYSTEM_PROMPT` を空文字列にしてください。

#### System Prompt Example

In [13]:
# System prompt
SYSTEM_PROMPT = "回答は常に、会話をさらに発展させるための批判的思考を促す一連の質問でなければなりません（質問自体に答えるのではなく）。ユーザーの質問自体に答えてはいけません。"
# SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "なぜ空は青いのか？"
# PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

興味深い観察ですね！この現象について一緒に考えてみましょう。

まず、あなたは「青い」と感じているものが、実際には何だと思いますか？空そのものに色があるのでしょうか、それとも私たちが見ているのは別の何かでしょうか？

そして、もし空が常に青いなら、なぜ夕焼けの時は赤やオレンジに見えるのでしょう？この違いは何を示唆していると思いますか？

また、光と色の関係について考えてみてください。太陽の光は本当は何色だと思いますか？そして、その光が地球の大気を通過する時に何が起こっていると想像できますか？


なぜシステムプロンプトを使うのでしょうか？ **良く書かれたシステムプロンプトは Claude の性能をさまざまな面で改善** できます（例：ルールや指示に従う能力の向上）。詳しくは、Claude での [システムプロンプトの使い方](https://docs.anthropic.com/claude/docs/how-to-use-system-prompts) のドキュメントを参照してください。

次は演習に進みます。上の内容を変えずにレッスンのプロンプトだけ試したい場合は、ノートブック末尾の [**Example Playground**](#example-playground) を利用してください。

---

## Exercises
- [Exercise 1.1 - Counting to Three](#exercise-11---counting-to-three)
- [Exercise 1.2 - System Prompt](#exercise-12---system-prompt)

### Exercise 1.1 - Counting to Three
正しい `user` / `assistant` 形式を使って、以下の `PROMPT` を編集し、Claude に **3まで数えさせて** ください。出力には、解答が正しいかどうかも表示されます。

In [16]:
# Prompt - this is the only field you should change
SYSTEM_PROMPT = "質問への回答のみを出力し、他は出力しないでください."
PROMPT = "1から3まで数えてください."
# PROMPT = "[Replace this text]"

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

1、2、3

--------------------------- GRADING ---------------------------
This exercise has been correctly solved: True


❓ ヒントが欲しい場合は、次のセルを実行してください！

In [15]:
print(hints.exercise_1_1_hint)

The grading function in this exercise is looking for an answer that contains the exact Arabic numerals "1", "2", and "3".
You can often get Claude to do what you want simply by asking.


### Exercise 1.2 - System Prompt

`SYSTEM_PROMPT` を変更して、Claude が **3歳の子ども** のように返答するようにしてください。

In [19]:
# System prompt - this is the only field you should change
SYSTEM_PROMPT = "3歳の子どものように返答してください."
# SYSTEM_PROMPT = "Answer like a 3 year old."
# SYSTEM_PROMPT = "[Replace this text]"

# Prompt
PROMPT = "空はどれくらい大きいの?"
# PROMPT = "How big is the sky?"

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search(r"ママ", text) or re.search(r"soo", text))
    # return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

わあ！そらってね、とーーーっても大きいの！

うーんとね、おそらのうえまでいくのに、ひこうきでもすっごく時間がかかるんだよ！

ママがね、「そらにはおわりがないんだよ」っていってたの。だから、どこまでいってもそらなの！

あのね、よるになると、そらにはおほしさまがいっぱいあるでしょ？あのおほしさまも、ぜーんぶそらのなかにあるんだって！

すごいよね！そらって、せかいでいちばん大きいとおもう！

ぼく（わたし）のおうちも、こうえんも、ぜーんぶそらのしたにあるんだもん！

--------------------------- GRADING ---------------------------
This exercise has been correctly solved: True


❓ ヒントが欲しい場合は、次のセルを実行してください！

In [20]:
print(hints.exercise_1_2_hint)

The grading function in this exercise is looking for answers that contain "soo" or "giggles".
There are many ways to solve this, just by asking!


### Congrats!

ここまでの演習をすべて解けたら、次の章に進む準備ができています。Happy prompting!

---

## Example Playground

このエリアでは、このレッスンで示したプロンプト例を自由に試し、プロンプトを調整して Claude の応答がどう変わるかを観察できます。

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Claude, how are you?"}
        ]
    )

# Print Claude's response
print(response[0].text)

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ]
    )

# Print Claude's response
print(response[0].text)

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))